We will begin with a few imports that we would need while implementing Naive Bayes

In [0]:
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import io
import random

#Step 1 Collecting Data
The data set of Zoo Animal Classification using Naive Bayes. The dataset can be downloaded from https://archive.ics.uci.edu/ml/datasets/Zoo

#Step 2 Exploring and preparing the data

## Data Preparation

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df = 'gdrive/My Drive/ALY_6020/zoo.csv'

col_names = ['animal name', 'hair', 'feathers', 'eggs', 'milk', 'airborne', 
               'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 
               'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize', 'type']
zoo_df = pd.read_csv(df, names=col_names)
zoo_df

,animal name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [0]:
"""
Convert leg attributes from [0,2,4,5,6,8] to 0-5
"""
def convert_legs_value(v):
  value_list = [0, 2, 4, 5, 6, 8]
  convert_list = [0, 1, 2, 3, 4, 5]
  return convert_list[value_list.index(v)]

zoo_df['legs'] = zoo_df['legs'].apply(convert_legs_value)
zoo_df

In [0]:
zoo_data = zoo_df.drop('animal name', 1).values.tolist()
zoo_data

[[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 4],
 [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 4],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 4],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
 [0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 4],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 7],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 7],
 [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 2],
 [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1,

## Data preparation - Create training + test dataset

In [0]:
def train_test_split(data, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(data))
    test_indices = random.sample(range(len(data)), k=test_size)
    train_indices = [i for i in range(len(data)) if i not in test_indices]
    
    test_df = []
    for index in test_indices:
        test_d = data[index]
        test_df.append(test_d)
    
    train_df = []
    for index in train_indices:
        train_d = data[index]
        train_df.append(train_d)
    
    return train_df, test_df

random.seed(0)
train_data, test_data = train_test_split(zoo_data, test_size = 30)
train_sample_num = len(train_data)
attr_num = len(train_data[0])-1

# 3.Traing a model on the data

In [0]:
"""
Split the dataset by class values, returns a dictionary
"""
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated
separated = separate_by_class(train_data)
separated

{1: [[1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 0, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1],
  [1, 0

In [0]:
"""
P(C = class)
"""
def calculate_class_prob(label):
    count = len(separated[label])
    py = count/train_sample_num
    return py

calculate_class_prob(3)


0.04225352112676056

In [0]:
"""
P(X = x |C = class)
"""
def calculate_feature_prob(label):
    label_rows = separated[label]
    label_num = len(label_rows)
    label_rows = separated[label]
    f_prob = []
    for i in range(attr_num):
        if i != 12:
            count0 = 0
            count1 = 0
            for row in label_rows:
                if row[i] == 0:
                    count0 += 1
                else:
                    count1 += 1
            cal = (count0/label_num, count1/label_num)
            f_prob.append(cal)
        else: # Leg features have 6 different values
            count0 = 0
            count1 = 0
            count2 = 0
            count3 = 0
            count4 = 0
            count5 = 0
            for row in label_rows:
                if row[i] == 0:
                    count0 += 1
                elif row[i] == 1:
                    count1 += 1
                elif row[i] == 2:
                    count2 += 1
                elif row[i] == 3:
                    count3 += 1
                elif row[i] == 4:
                    count4 += 1
                else:
                    count5 += 1
            cal = (count0/label_num, count1/label_num, count2/label_num,
                   count3/label_num, count4/label_num, count5/label_num)
            f_prob.append(cal)
            
    return f_prob

calculate_feature_prob(1)

[(0.06896551724137931, 0.9310344827586207),
 (1.0, 0.0),
 (0.9655172413793104, 0.034482758620689655),
 (0.0, 1.0),
 (0.9655172413793104, 0.034482758620689655),
 (0.8275862068965517, 0.1724137931034483),
 (0.41379310344827586, 0.5862068965517241),
 (0.034482758620689655, 0.9655172413793104),
 (0.0, 1.0),
 (0.0, 1.0),
 (1.0, 0.0),
 (0.896551724137931, 0.10344827586206896),
 (0.06896551724137931, 0.1724137931034483, 0.7586206896551724, 0.0, 0.0, 0.0),
 (0.10344827586206896, 0.896551724137931),
 (0.8275862068965517, 0.1724137931034483),
 (0.1724137931034483, 0.8275862068965517)]

In [0]:
separated[3]

[[0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 0, 1, 3],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 3]]

In [0]:
def create_model():
    model = {}
    for class_value in separated:
        pxy = calculate_feature_prob(class_value)
        py = calculate_class_prob(class_value)
        model[class_value] = [pxy, py]
    return model

model = create_model()
model

{1: [[(0.06896551724137931, 0.9310344827586207),
   (1.0, 0.0),
   (0.9655172413793104, 0.034482758620689655),
   (0.0, 1.0),
   (0.9655172413793104, 0.034482758620689655),
   (0.8275862068965517, 0.1724137931034483),
   (0.41379310344827586, 0.5862068965517241),
   (0.034482758620689655, 0.9655172413793104),
   (0.0, 1.0),
   (0.0, 1.0),
   (1.0, 0.0),
   (0.896551724137931, 0.10344827586206896),
   (0.06896551724137931,
    0.1724137931034483,
    0.7586206896551724,
    0.0,
    0.0,
    0.0),
   (0.10344827586206896, 0.896551724137931),
   (0.8275862068965517, 0.1724137931034483),
   (0.1724137931034483, 0.8275862068965517)],
  0.4084507042253521],
 2: [[(1.0, 0.0),
   (0.0, 1.0),
   (0.0, 1.0),
   (1.0, 0.0),
   (0.1875, 0.8125),
   (0.75, 0.25),
   (0.625, 0.375),
   (1.0, 0.0),
   (0.0, 1.0),
   (0.0, 1.0),
   (1.0, 0.0),
   (1.0, 0.0),
   (0.0, 1.0, 0.0, 0.0, 0.0, 0.0),
   (0.0, 1.0),
   (0.8125, 0.1875),
   (0.6875, 0.3125)],
  0.22535211267605634],
 3: [[(1.0, 0.0),
   (1.0, 

In [0]:
def evaluate_class_probabilities(model, sample):
    probabilities = dict()
    for class_value, class_summaries in model.items():
        probabilities[class_value] = class_summaries[1]
        for i in range(attr_num):
            probabilities[class_value] *= class_summaries[0][i][int(sample[i])]
    return probabilities

evaluate_class_probabilities(model, test_data[0])

{1: 0.014448085944456403, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}

In [0]:
"""
Predict the class for a given sample
"""
def predict(model, sample):
    probabilities = evaluate_class_probabilities(model, sample)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

predict(model, test_data[1])

1

# 4. Evaluate model performance

In [0]:
"""
Evaluate method
"""
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [0]:
# get all the predictions
predictions = []
for row in test_data:
    best_label = predict(model, row)
    predictions.append(best_label)

# get all the real labels
actual = []
for row in test_data:
    real_label = row[-1]
    actual.append(real_label)
    
# Calculate accuracy percentage
accuracy = accuracy_metric(actual, predictions)
print("The accuracy of predcition is: %.2f percent!" %accuracy )

The accuracy of predcition is: 76.67 percent!


#5. Improving model performance

In [0]:
"""
Calculate prob with laplace estimator
"""
laplace_estimator = 1

def evaluate_class_probabilities_2(model, sample):
    probabilities = dict()
    for class_value, class_summaries in model.items():
        probabilities[class_value] = class_summaries[1]
        for i in range(attr_num):
          f_prob = class_summaries[0][i][int(sample[i])]
          if f_prob == 0:
            f_prob = laplace_estimator / len(separated[class_value])
          probabilities[class_value] *= f_prob
    return probabilities

evaluate_class_probabilities(model, test_data[0])

{1: 0.014448085944456403, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}

In [0]:
"""
P(X = x |C = class) with laplace
"""
laplace_estimator = 1

def calc_prob(c, sample_size):
  return c / sample_size if c > 0 else laplace_estimator / sample_size

def calculate_feature_prob_2(label):
  label_rows = separated[label]
  label_num = len(label_rows)
  label_rows = separated[label]
  f_prob = []
  for i in range(attr_num):
      if i != 12:
          count0 = 0
          count1 = 0
          for row in label_rows:
              if row[i] == 0:
                  count0 += 1
              else:
                  count1 += 1
          cal = (
              calc_prob(count0, label_num),
              calc_prob(count1, label_num)
          )
          f_prob.append(cal)
      else: # Leg features have 6 different values
          count0 = 0
          count1 = 0
          count2 = 0
          count3 = 0
          count4 = 0
          count5 = 0
          for row in label_rows:
              if row[i] == 0:
                  count0 += 1
              elif row[i] == 1:
                  count1 += 1
              elif row[i] == 2:
                  count2 += 1
              elif row[i] == 3:
                  count3 += 1
              elif row[i] == 4:
                  count4 += 1
              else:
                  count5 += 1
          cal = (
              calc_prob(count0, label_num),
              calc_prob(count1, label_num),
              calc_prob(count2, label_num),
              calc_prob(count3, label_num),
              calc_prob(count4, label_num),
              calc_prob(count5, label_num)
          )
          f_prob.append(cal)
          
  return f_prob

calculate_feature_prob_2(1)

[(0.06896551724137931, 0.9310344827586207),
 (1.0, 0.034482758620689655),
 (0.9655172413793104, 0.034482758620689655),
 (0.034482758620689655, 1.0),
 (0.9655172413793104, 0.034482758620689655),
 (0.8275862068965517, 0.1724137931034483),
 (0.41379310344827586, 0.5862068965517241),
 (0.034482758620689655, 0.9655172413793104),
 (0.034482758620689655, 1.0),
 (0.034482758620689655, 1.0),
 (1.0, 0.034482758620689655),
 (0.896551724137931, 0.10344827586206896),
 (0.06896551724137931,
  0.1724137931034483,
  0.7586206896551724,
  0.034482758620689655,
  0.034482758620689655,
  0.034482758620689655),
 (0.10344827586206896, 0.896551724137931),
 (0.8275862068965517, 0.1724137931034483),
 (0.1724137931034483, 0.8275862068965517)]

In [0]:
def create_model_2():
    model = {}
    for class_value in separated:
        pxy = calculate_feature_prob_2(class_value)
        py = calculate_class_prob(class_value)
        model[class_value] = [pxy, py]
    return model

model_2 = create_model_2()
model_2

{1: [[(0.06896551724137931, 0.9310344827586207),
   (1.0, 0.034482758620689655),
   (0.9655172413793104, 0.034482758620689655),
   (0.034482758620689655, 1.0),
   (0.9655172413793104, 0.034482758620689655),
   (0.8275862068965517, 0.1724137931034483),
   (0.41379310344827586, 0.5862068965517241),
   (0.034482758620689655, 0.9655172413793104),
   (0.034482758620689655, 1.0),
   (0.034482758620689655, 1.0),
   (1.0, 0.034482758620689655),
   (0.896551724137931, 0.10344827586206896),
   (0.06896551724137931,
    0.1724137931034483,
    0.7586206896551724,
    0.034482758620689655,
    0.034482758620689655,
    0.034482758620689655),
   (0.10344827586206896, 0.896551724137931),
   (0.8275862068965517, 0.1724137931034483),
   (0.1724137931034483, 0.8275862068965517)],
  0.4084507042253521],
 2: [[(1.0, 0.0625),
   (0.0625, 1.0),
   (0.0625, 1.0),
   (1.0, 0.0625),
   (0.1875, 0.8125),
   (0.75, 0.25),
   (0.625, 0.375),
   (1.0, 0.0625),
   (0.0625, 1.0),
   (0.0625, 1.0),
   (1.0, 0.0625),

In [0]:
# get all the predictions
predictions = []
for row in test_data:
    best_label = predict(model_2, row)
    predictions.append(best_label)

# get all the real labels
actual = []
for row in test_data:
    real_label = row[-1]
    actual.append(real_label)
    
# Calculate accuracy percentage
accuracy = accuracy_metric(actual, predictions)
print("The accuracy of predcition is: %.2f percent!" %accuracy )

The accuracy of predcition is: 96.67 percent!
